In [1]:
#!/usr/bin/env python3
# Copyright 2022 ETH Zurich and University of Bologna.
# Licensed under the Apache License, Version 2.0, see LICENSE for details.
# SPDX-License-Identifier: Apache-2.0

import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import argparse
import pathlib
import hjson
import random
import os

In [2]:
np.random.seed(42)
torch.manual_seed(42)
global verbose

In [3]:
def array_to_cstr(a):
    out = '{'
    if isinstance(a, np.ndarray):
        a = a.flat
    if isinstance(a, torch.Tensor):
        a = a.numpy().flat
    for el in a:
        out += '{}, '.format(el)
    out = out[:-2] + '}'
    return out

In [208]:
# TODO: check what is missing for CNN
def emit_mnist_cnn_data(name='mnist_cnn', **kwargs):
    
    # constants
    ## output channels
    co = kwargs['CO']
    ## input channels
    ci = kwargs['CI']
    ## input height 
    h = kwargs['H']
    ## input width
    w = kwargs['W']
    ## filter dimension (assuming square)
    k = kwargs['K']
    ## padding
    p = kwargs['padding']
    ## stride
    s = kwargs['stride']
    
    # data
    MAT_INPUT = kwargs['INPUT']
    MAT_CONV1_WEIGHTS = kwargs['CONV1_WEIGHTS']
    MAT_CONV1_BIAS = kwargs['CONV1_BIAS']


    layer_str = ''
    layer_str += '#include "network.h"\n\n'
    layer_str += f'cnn_t {name}_t = {{\n'
    layer_str += f'\t.CO = {co},\n'
    layer_str += f'\t.CI = {ci},\n'
    layer_str += f'\t.H = {h},\n'
    layer_str += f'\t.W = {w},\n'
    layer_str += f'\t.K = {k},\n'
    layer_str += f'\t.padding = {p},\n'
    layer_str += f'\t.stride = {s},\n'
    layer_str += f'\t.dtype = FP{kwargs["prec"]}\n'
    layer_str += '};\n\n\n'

    ctypes = {
        '64': 'double',
        '32': 'float',
        '16': '__fp16',
        'B16': '__bf16',
        '8': 'char'
    }

    dtype = ctypes[str(kwargs['prec'])]

    # network initialization parameters
    layer_str += f'static {dtype} {name}_conv1_weights_dram [{co}][{ci}][{k}][{k}] = ' + array_to_cstr(MAT_CONV1_WEIGHTS) + ';\n\n\n'
    layer_str += f'static {dtype} {name}_conv1_bias_dram [{co}] = ' + array_to_cstr(MAT_CONV1_BIAS) + ';\n\n\n'

    # input data
    layer_str += f'static {dtype} {name}_image_dram [{w*h}][{1}] = ' + array_to_cstr(MAT_INPUT) + ';\n\n\n'

    return layer_str


In [209]:
def emit_mnist_header_file(layer_type: str, **kwargs):

    file_path = '/scratch/msc22f11/msc22f11/snitch/sw/applications/data/'
    emit_str = "// Copyright 2022 ETH Zurich and University of Bologna.\n" + \
               "// Licensed under the Apache License, Version 2.0, see LICENSE for details.\n" + \
               "// SPDX-License-Identifier: Apache-2.0\n\n"

    if(layer_type == 'mnist_cnn'):
        file = file_path + 'data_cnn_mnist.h'
        emit_str += emit_mnist_cnn_data(**kwargs)

    with open(file, 'w') as f:
        f.write(emit_str)

In [4]:
# download MNIST dataset using DataLoader

transform = transforms.Compose(
    [
        transforms.ToTensor()
    ]
)

PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
mnist_dataset = MNIST(PATH_DATASETS, train=True, transform=transform, download=True)

# set seeds for reproducability 
g = torch.Generator()
g.manual_seed(42)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

mnist_dl = DataLoader(mnist_dataset, worker_init_fn=seed_worker, generator=g)

In [5]:
print(mnist_dataset)

Dataset MNIST
    Number of datapoints: 60000
    Root location: .
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
           )


In [216]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        torch.manual_seed(42)
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=16,            
                kernel_size=5,              
                stride=1,                   
                padding=2,                  
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
        torch.manual_seed(42)
        self.conv2 = nn.Sequential(         
            nn.Conv2d(16, 32, 5, 1, 2),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        # fully connected layer, output 10 classes
        torch.manual_seed(42)
        self.out = nn.Linear(32 * 7 * 7, 10)
        
    def forward(self, x):
        print(x.shape)
        x = self.conv1(x)
        print(x.shape)
        x = self.conv2(x)
        print(x.shape)
        # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
        x = x.view(x.size(0), -1)
        print(x.shape)       
        output = self.out(x)
        print(output)
        return output, x    # return x for visualization

In [217]:
net = CNN()
print(net)

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)


In [9]:
"""
Now we iterate through a smaller subset of the dataset 
to retrieve the image data with their
respective labels
"""

data_iterator = iter(mnist_dl)

for i in range(0, 5):
    s_image, s_label = data_iterator.next()
    np_s_image = s_image.numpy().flatten()
    np_s_label = s_label.numpy().flatten()
    if(i==0):
        s_images = np.array(np_s_image.tolist())
        s_labels = np.array(np_s_label.tolist())
    else:
        s_images = np.append(s_images, np_s_image)
        s_labels = np.append(s_labels, np_s_label)

In [218]:
first_im, first_label = next(iter(mnist_dl))

In [219]:
torch.manual_seed(42)

weights_conv1 = net.conv1[0].weight
print('Shape of CONV1 weights: ', weights_conv1.shape)
biases_conv1 = net.conv1[0].bias
print('Shape of CONV1 biases: ', biases_conv1.shape)
weights_conv2 = net.conv2[0].weight
biases_conv2 = net.conv2[0].bias

criterion = nn.CrossEntropyLoss()

for i in range(1):
    net.zero_grad
    output = net(first_im)[0]
    loss = criterion(output, first_label)
    loss.backward()
    weight_grads_conv1 = net.conv1[0].weight.grad
    bias_grads_conv1 = net.conv1[0].bias.grad
    weight_grads_conv2 = net.conv2[0].weight.grad
    bias_grads_conv2 = net.conv2[0].bias.grad


Shape of CONV1 weights:  torch.Size([16, 1, 5, 5])
Shape of CONV1 biases:  torch.Size([16])
torch.Size([1, 1, 28, 28])
torch.Size([1, 16, 14, 14])
torch.Size([1, 32, 7, 7])
torch.Size([1, 1568])
tensor([[-0.1005,  0.0660, -0.0802,  0.0697, -0.0005,  0.0110, -0.0027, -0.0046,
          0.0114,  0.0355]], grad_fn=<AddmmBackward0>)


In [227]:
kwargs = {
    'CO': 16,
    'CI': 1,
    'H': 28,
    'W': 28,
    'K': 5,
    'padding': 2,
    'stride': 1,
    'INPUT': first_im.numpy().flatten(),
    'CONV1_WEIGHTS': weights_conv1.detach().numpy().flatten(),
    'CONV1_BIAS': biases_conv1.detach().numpy().flatten(),
    'prec': 64
}

In [228]:
emit_mnist_header_file('mnist_cnn', **kwargs)

In [66]:
print(first_im.shape)
net.conv1[0](first_im).shape

torch.Size([1, 1, 28, 28])


torch.Size([1, 16, 28, 28])

In [68]:
print(weights_conv1.shape)
print(biases_conv1.shape)

torch.Size([16, 1, 5, 5])
torch.Size([16])


In [70]:
weights_conv1[0].shape

torch.Size([1, 5, 5])

In [222]:
from torchvision import models
from torchsummary import summary
summary(net, (1, 28, 28))

torch.Size([2, 1, 28, 28])
torch.Size([2, 16, 14, 14])
torch.Size([2, 32, 7, 7])
torch.Size([2, 1568])
tensor([[ 0.0099,  0.0543, -0.0675,  0.1394, -0.0166, -0.0084, -0.0534, -0.1296,
          0.0663, -0.0227],
        [ 0.0056,  0.0306, -0.0367,  0.1300,  0.0285, -0.0247, -0.1016, -0.1288,
          0.0791, -0.0425]], grad_fn=<AddmmBackward0>)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             416
              ReLU-2           [-1, 16, 28, 28]               0
         MaxPool2d-3           [-1, 16, 14, 14]               0
            Conv2d-4           [-1, 32, 14, 14]          12,832
              ReLU-5           [-1, 32, 14, 14]               0
         MaxPool2d-6             [-1, 32, 7, 7]               0
            Linear-7                   [-1, 10]          15,690
Total params: 28,938
Trainable params: 28,938
Non-trainable params: 0
-----

In [110]:
biases_conv1.detach().numpy()

array([-0.05595953,  0.13507354,  0.01593194,  0.00902367, -0.04922011,
       -0.18111794, -0.18805149, -0.09560301, -0.10166428,  0.06231072,
       -0.05822215, -0.07824443,  0.19068597,  0.06966458,  0.14258046,
       -0.09682255], dtype=float32)

In [212]:
net.conv1[0](first_im).shape

torch.Size([1, 16, 28, 28])

In [129]:
net.conv1[0](first_im)[0][0].detach().numpy()[3]

array([-0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
       -0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
       -0.05781687, -0.06554271, -0.05869401, -0.06374829, -0.13307634,
       -0.08849649, -0.14184302, -0.04242297, -0.25899282, -0.20098922,
       -0.1947165 , -0.11197067, -0.15200649, -0.21128216, -0.15689726,
       -0.10185251, -0.05595953, -0.05595953], dtype=float32)

In [131]:
(first_conv_img + biases_conv1[0].detach().numpy())[4]

array([-0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
       -0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
       -0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
       -0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
       -0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
       -0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
       -0.05595953, -0.05595953])

In [133]:
first_im[0][0].shape

torch.Size([28, 28])

In [195]:
first_im.shape

torch.Size([1, 1, 28, 28])

In [196]:
# image dimensions
w_in = first_im[0][0].numpy().shape[0]
h_in = first_im[0][0].numpy().shape[1]
# input channel(s)
c_in = 1
print("Input Feature Map: {}x{}x{}x{}".format(1, c_in, w_in, h_in))
# output channels
c_out = 16
# kernel size
k = 5
# stride
s = 1
# padding
p = 2
# output dimensions
w_out = (w_in - k + 2 * p) // s + 1
h_out = (h_in - k + 2 * p) // s + 1
print("Output Feature Map: {}x{}x{}x{}".format(1, c_out, w_out, h_out))

Input Feature Map: 1x1x28x28
Output Feature Map: 1x16x28x28


In [172]:
# pad the image data with zeros
# pad_width = ((2,2), (2,2))
# first_im[0][0].numpy() = np.pad(first_im[0][0].numpy(), pad_width, 'constant', constant_values=0)
# img_padded = np.pad(first_im[0][0].numpy(), ((2,2), (2,2)), 'constant', constant_values=0)
# img_padded.shape
img_padded = F.pad(first_im, (2, 2, 2, 2))
img_padded.shape

torch.Size([1, 1, 32, 32])

In [254]:
result = np.zeros(shape=(1, c_out, w_out, h_out))
for o1 in range(w_out):
    for o2 in range(h_out):
        for co in range(c_out):
            total = 0
            for ci in range(c_in):
                kt = 0
                for kh in range(k):
                    for kw in range(k):
                        weight = weights_conv1[co][ci][kh][kw].detach().numpy()
                        pos1 = kh + o1 * s
                        pos2 = kw + o2 * s
                        value = img_padded[0][ci][pos1][pos2].numpy()
                        kt += weight * value
                total += kt
            result[0][co][o1][o2] = total + biases_conv1[co].detach().numpy()


In [255]:
result2 = np.zeros(shape=(1, c_out, w_out, h_out))
for co in range(c_out):
    for o1 in range(w_out):
        for o2 in range(h_out):
            total = 0
            for ci in range(c_in):
                kt = 0
                for kh in range(k):
                    for kw in range(k):
                        weight = weights_conv1[co][ci][kh][kw].detach().numpy()
                        pos1 = kh + o1 * s
                        pos2 = kw + o2 * s
                        value = img_padded[0][ci][pos1][pos2].numpy()
                        kt += weight * value
                total += kt
            result2[0][co][o1][o2] = total + biases_conv1[co].detach().numpy()

In [313]:
# test with single kernel only 
result2 = np.zeros(shape=(1, 1, w_out, h_out))
for co in range(1):
    for o1 in range(w_out):
        for o2 in range(h_out):
            total = 0
            for ci in range(c_in):
                kt = 0
                for kh in range(k):
                    for kw in range(k):
                        weight = weights_conv1[co][ci][kh][kw].detach().numpy()
                        pos1 = kh + o1 * s
                        pos2 = kw + o2 * s
                        value = img_padded[0][ci][pos1][pos2].numpy()
                        if value != 0:
                            print("data[{}][{}] = {}".format(pos1, pos2, value))
                            print("weight[{}][{}] = {}".format(kh, kw, weight))
                        kt += weight * value
                total += kt
                # print("total = {}".format(total))
            result2[0][co][o1][o2] = total + biases_conv1[co].detach().numpy()

data[7][14] = 0.0117647061124444
weight[4][4] = -0.15787403285503387
data[7][14] = 0.0117647061124444
weight[4][3] = 0.1326742023229599
data[7][15] = 0.07058823853731155
weight[4][4] = -0.15787403285503387
data[7][14] = 0.0117647061124444
weight[4][2] = -0.08123166859149933
data[7][15] = 0.07058823853731155
weight[4][3] = 0.1326742023229599
data[7][16] = 0.07058823853731155
weight[4][4] = -0.15787403285503387
data[7][14] = 0.0117647061124444
weight[4][1] = -0.0234545711427927
data[7][15] = 0.07058823853731155
weight[4][2] = -0.08123166859149933
data[7][16] = 0.07058823853731155
weight[4][3] = 0.1326742023229599
data[7][17] = 0.07058823853731155
weight[4][4] = -0.15787403285503387
data[7][14] = 0.0117647061124444
weight[4][0] = -0.0921473279595375
data[7][15] = 0.07058823853731155
weight[4][1] = -0.0234545711427927
data[7][16] = 0.07058823853731155
weight[4][2] = -0.08123166859149933
data[7][17] = 0.07058823853731155
weight[4][3] = 0.1326742023229599
data[7][18] = 0.4941176474094391
wei

In [306]:
weights_conv1[0][0].detach().numpy()

array([[ 0.15290771,  0.16600159, -0.0468545 ,  0.18372226, -0.04382072],
       [ 0.04035814, -0.097371  ,  0.11745653,  0.17630859, -0.14672564],
       [ 0.17383924,  0.03743186,  0.14776178,  0.02708611,  0.09643763],
       [-0.0282382 ,  0.15417717,  0.02956178, -0.09336799,  0.05097966],
       [-0.09214733, -0.02345457, -0.08123167,  0.1326742 , -0.15787403]],
      dtype=float32)

In [308]:
weights_conv1[0][0].detach().numpy()[4][4]

-0.15787403

In [307]:
weights_conv1[0][0].detach().numpy().flatten()

array([ 0.15290771,  0.16600159, -0.0468545 ,  0.18372226, -0.04382072,
        0.04035814, -0.097371  ,  0.11745653,  0.17630859, -0.14672564,
        0.17383924,  0.03743186,  0.14776178,  0.02708611,  0.09643763,
       -0.0282382 ,  0.15417717,  0.02956178, -0.09336799,  0.05097966,
       -0.09214733, -0.02345457, -0.08123167,  0.1326742 , -0.15787403],
      dtype=float32)

In [309]:
weights_conv1[0][0].detach().numpy().flatten()[4 * 5 + 4]

-0.15787403

In [283]:
img_padded[0][0].flatten()

tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [297]:
img_padded[0][0][7][14]

tensor(0.0118)

In [299]:
img_padded[0][0].flatten().numpy()[7 * 32 + 14]

0.011764706

In [296]:
7 * 28 + 14 * 2

224

In [291]:
np.where(img_padded[0][0].flatten().numpy() == 0.011764706)

(array([238]),)

In [310]:
result2.shape

(1, 1, 28, 28)

In [311]:
result2[0][0].shape

(28, 28)

In [262]:
weights_conv1[0][0][0][1].detach().numpy()


array(0.16600159, dtype=float32)

In [256]:
result.shape

(1, 16, 28, 28)

In [257]:
result2.shape

(1, 16, 28, 28)

In [233]:
img_padded[0][0][1][1]

tensor(0.)

In [179]:
net.conv1[0](first_im).shape

torch.Size([1, 16, 28, 28])

In [264]:
result[0][0][3]

array([-0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
       -0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
       -0.05781688, -0.06554271, -0.05869401, -0.06374828, -0.13307635,
       -0.08849649, -0.14184302, -0.04242296, -0.25899283, -0.20098922,
       -0.19471649, -0.11197067, -0.15200648, -0.21128216, -0.15689726,
       -0.10185252, -0.05595953, -0.05595953])

In [259]:
result2[0][0][3]

array([-0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
       -0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
       -0.05781688, -0.06554271, -0.05869401, -0.06374828, -0.13307635,
       -0.08849649, -0.14184302, -0.04242296, -0.25899283, -0.20098922,
       -0.19471649, -0.11197067, -0.15200648, -0.21128216, -0.15689726,
       -0.10185252, -0.05595953, -0.05595953])

In [260]:
net.conv1[0](first_im)[0][0].detach().numpy()[3]

array([-0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
       -0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
       -0.05781687, -0.06554271, -0.05869401, -0.06374829, -0.13307634,
       -0.08849649, -0.14184302, -0.04242297, -0.25899282, -0.20098922,
       -0.1947165 , -0.11197067, -0.15200649, -0.21128216, -0.15689726,
       -0.10185251, -0.05595953, -0.05595953], dtype=float32)

# Playground

In [226]:
weights_conv1[0]

tensor([[[ 0.1529,  0.1660, -0.0469,  0.1837, -0.0438],
         [ 0.0404, -0.0974,  0.1175,  0.1763, -0.1467],
         [ 0.1738,  0.0374,  0.1478,  0.0271,  0.0964],
         [-0.0282,  0.1542,  0.0296, -0.0934,  0.0510],
         [-0.0921, -0.0235, -0.0812,  0.1327, -0.1579]]],
       grad_fn=<SelectBackward0>)